# Platypus 2 w/o person

## 7B - Context window 1

In [4]:
# Import
import os
os.environ['TRANSFORMERS_CACHE'] = "cache_dir"
import transformers
from transformers import pipeline
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import re
import pandas as pd
import torch
import langchain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import sklearn
import time
from sacrebleu import BLEU
import tqdm.contrib
# Load data
df = pd.read_csv("./data/KnowledgeTransfers-one_sent.csv", sep=",")
# Load model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "garage-bAInd/Platypus2-7B"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model_bf16 = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config)
# Run pipeline
def run_llm(snippets):
    outputs = []
    for snippet in zip(snippets):
        pipe = pipeline(
        "text-generation", model=model_bf16, tokenizer=tokenizer, max_new_tokens=200, temperature = 0.001
        )
        hf = HuggingFacePipeline(pipeline=pipe)
        template = """
        ####
        Instruction: You are a literary scholar.
        What is the family relation in the German text {drama_snippet}?
        The possible family relations are parent, child, uncle, siblings, cousins.
        Answer in a single sentence in the following format: The family relation is >>correct family relation<<.
        Do NOT write code.
        Do NOT write anything before or after the answer sentence.
        ####
        """
        drama_snippet = str(snippet)
        prompt = PromptTemplate(
            input_variables=["drama_snippet"],
            template=template,
        )
        chain = LLMChain(llm=hf, prompt=prompt)
        output = chain.run({"drama_snippet": {drama_snippet}})
        outputs.append(output)
    return outputs
df_sample = df.loc[(df.annotation != "no-transfer") & (df.relation.isin(["parent_of", "child_of", "siblings", "uncle_of", "cousins"]))]
llm_outputs = run_llm(df_sample.context)
# Evaluate
predictions_relation = []
for o in llm_outputs:
    if "parent" in o:
        prediction_relation = ["parent_of"]
    elif "child" in o:
        prediction_relation = ["child_of"]
    elif "siblings" in o:
        prediction_relation = ["siblings"]
    elif "uncle" in o:
        prediction_relation = ["uncle_of"]
    elif "cousins" in o:
        prediction_relation = ["cousins"]
    else:
        prediction_relation = ["else"]
    predictions_relation.extend(prediction_relation)
results_relation = pd.DataFrame(list(zip(list(df_sample.relation), predictions_relation)), columns = ["key", "pred"])
pd.DataFrame(list(zip([sklearn.metrics.f1_score(results_relation.key, results_relation.pred, average="weighted")], 
             [sklearn.metrics.recall_score(results_relation.key, results_relation.pred, average="weighted")], 
             [sklearn.metrics.precision_score(results_relation.key, results_relation.pred, average="weighted")],
             [sklearn.metrics.accuracy_score(results_relation.key, results_relation.pred)])), 
             columns = ["F1", "Recall", "Precision", "Accuracy"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F1,Recall,Precision,Accuracy
0,0.369612,0.369048,0.470545,0.369048


## 7B - Context window 2

In [3]:
# Import
import os
os.environ['TRANSFORMERS_CACHE'] = "cache_dir"
import transformers
from transformers import pipeline
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import re
import pandas as pd
import torch
import langchain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import sklearn
import time
from sacrebleu import BLEU
import tqdm.contrib
# Load data
df = pd.read_csv("./data/KnowledgeTransfers-two_sent.csv", sep=",")
# Load model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "garage-bAInd/Platypus2-7B"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model_bf16 = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config)
# Run pipeline
def run_llm(snippets):
    outputs = []
    for snippet in zip(snippets):
        pipe = pipeline(
        "text-generation", model=model_bf16, tokenizer=tokenizer, max_new_tokens=200, temperature = 0.001
        )
        hf = HuggingFacePipeline(pipeline=pipe)
        template = """
        ####
        Instruction: You are a literary scholar.
        What is the family relation in the German text {drama_snippet}?
        The possible family relations are parent, child, uncle, siblings, cousins.
        Answer in a single sentence in the following format: The family relation is >>correct family relation<<.
        Do NOT write code.
        Do NOT write anything before or after the answer sentence.
        ####
        """
        drama_snippet = str(snippet)
        prompt = PromptTemplate(
            input_variables=["drama_snippet"],
            template=template,
        )
        chain = LLMChain(llm=hf, prompt=prompt)
        output = chain.run({"drama_snippet": {drama_snippet}})
        outputs.append(output)
    return outputs
df_sample = df.loc[(df.annotation != "no-transfer") & (df.relation.isin(["parent_of", "child_of", "siblings", "uncle_of", "cousins"]))]
llm_outputs = run_llm(df_sample.context)
# Evaluate
predictions_relation = []
for o in llm_outputs:
    if "parent" in o:
        prediction_relation = ["parent_of"]
    elif "child" in o:
        prediction_relation = ["child_of"]
    elif "siblings" in o:
        prediction_relation = ["siblings"]
    elif "uncle" in o:
        prediction_relation = ["uncle_of"]
    elif "cousins" in o:
        prediction_relation = ["cousins"]
    else:
        prediction_relation = ["else"]
    predictions_relation.extend(prediction_relation)
results_relation = pd.DataFrame(list(zip(list(df_sample.relation), predictions_relation)), columns = ["key", "pred"])
pd.DataFrame(list(zip([sklearn.metrics.f1_score(results_relation.key, results_relation.pred, average="weighted")], 
             [sklearn.metrics.recall_score(results_relation.key, results_relation.pred, average="weighted")], 
             [sklearn.metrics.precision_score(results_relation.key, results_relation.pred, average="weighted")],
             [sklearn.metrics.accuracy_score(results_relation.key, results_relation.pred)])), 
             columns = ["F1", "Recall", "Precision", "Accuracy"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F1,Recall,Precision,Accuracy
0,0.269127,0.25,0.462868,0.25


## 13B - Context window 1

In [2]:
# Import
import os
os.environ['TRANSFORMERS_CACHE'] = "cache_dir"
import transformers
from transformers import pipeline
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import re
import pandas as pd
import torch
import langchain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import sklearn
import time
from sacrebleu import BLEU
import tqdm.contrib
# Load data
df = pd.read_csv("./data/KnowledgeTransfers-one_sent.csv", sep=",")
# Load model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "garage-bAInd/Platypus2-13B"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model_bf16 = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config)
# Run pipeline
def run_llm(snippets):
    outputs = []
    for snippet in zip(snippets):
        pipe = pipeline(
        "text-generation", model=model_bf16, tokenizer=tokenizer, max_new_tokens=200, temperature = 0.001
        )
        hf = HuggingFacePipeline(pipeline=pipe)
        template = """
        ####
        Instruction: You are a literary scholar.
        What is the family relation in the German text {drama_snippet}?
        The possible family relations are parent, child, uncle, siblings, cousins.
        Answer in a single sentence in the following format: The family relation is >>correct family relation<<.
        Do NOT write code.
        Do NOT write anything before or after the answer sentence.
        ####
        """
        drama_snippet = str(snippet)
        prompt = PromptTemplate(
            input_variables=["drama_snippet"],
            template=template,
        )
        chain = LLMChain(llm=hf, prompt=prompt)
        output = chain.run({"drama_snippet": {drama_snippet}})
        outputs.append(output)
    return outputs
df_sample = df.loc[(df.annotation != "no-transfer") & (df.relation.isin(["parent_of", "child_of", "siblings", "uncle_of", "cousins"]))]
llm_outputs = run_llm(df_sample.context)
# Evaluate
predictions_relation = []
for o in llm_outputs:
    if "parent" in o:
        prediction_relation = ["parent_of"]
    elif "child" in o:
        prediction_relation = ["child_of"]
    elif "siblings" in o:
        prediction_relation = ["siblings"]
    elif "uncle" in o:
        prediction_relation = ["uncle_of"]
    elif "cousins" in o:
        prediction_relation = ["cousins"]
    predictions_relation.extend(prediction_relation)
results_relation = pd.DataFrame(list(zip(list(df_sample.relation), predictions_relation)), columns = ["key", "pred"])
pd.DataFrame(list(zip([sklearn.metrics.f1_score(results_relation.key, results_relation.pred, average="weighted")], 
             [sklearn.metrics.recall_score(results_relation.key, results_relation.pred, average="weighted")], 
             [sklearn.metrics.precision_score(results_relation.key, results_relation.pred, average="weighted")],
             [sklearn.metrics.accuracy_score(results_relation.key, results_relation.pred)])), 
             columns = ["F1", "Recall", "Precision", "Accuracy"])

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


,F1,Recall,Precision,Accuracy
0,0.442883,0.5,0.514103,0.5


## 13B - context window 2

In [1]:
# Import
import os
os.environ['TRANSFORMERS_CACHE'] = "cache_dir"
import transformers
from transformers import pipeline
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import re
import pandas as pd
import torch
import langchain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import sklearn
import time
from sacrebleu import BLEU
import tqdm.contrib
# Load data
df = pd.read_csv("./data/KnowledgeTransfers-two_sent.csv", sep=",")
# Load model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_id = "garage-bAInd/Platypus2-13B"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model_bf16 = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config)











def run_llm(snippets):
    outputs = []
    for snippet in zip(snippets):
        pipe = pipeline(
        "text-generation", model=model_bf16, tokenizer=tokenizer, max_new_tokens=200, temperature = 0.001
        )
        hf = HuggingFacePipeline(pipeline=pipe)
        template = """
        ####
        Instruction: You are a literary scholar.
        What is the family relation in the German text {drama_snippet}?
        The possible family relations are parent, child, uncle, siblings, cousins.
        Answer in a single sentence in the following format: The family relation is >>correct family relation<<.
        Do NOT write code.
        Do NOT write anything before or after the answer sentence.
        ####
        """
        drama_snippet = str(snippet)
        prompt = PromptTemplate(
            input_variables=["drama_snippet"],
            template=template,
        )
        chain = LLMChain(llm=hf, prompt=prompt)
        output = chain.run({"drama_snippet": {drama_snippet}})
        outputs.append(output)
    return outputs
df_sample = df.loc[(df.annotation != "no-transfer") & (df.relation.isin(["parent_of", "child_of", "siblings", "uncle_of", "cousins"]))]
llm_outputs = run_llm(df_sample.context)
# Evaluate
predictions_relation = []
for o in llm_outputs:
    if "parent" in o:
        prediction_relation = ["parent_of"]
    elif "child" in o:
        prediction_relation = ["child_of"]
    elif "siblings" in o:
        prediction_relation = ["siblings"]
    elif "uncle" in o:
        prediction_relation = ["uncle_of"]
    elif "cousins" in o:
        prediction_relation = ["cousins"]
    predictions_relation.extend(prediction_relation)
results_relation = pd.DataFrame(list(zip(list(df_sample.relation), predictions_relation)), columns = ["key", "pred"])
pd.DataFrame(list(zip([sklearn.metrics.f1_score(results_relation.key, results_relation.pred, average="weighted")], 
             [sklearn.metrics.recall_score(results_relation.key, results_relation.pred, average="weighted")], 
             [sklearn.metrics.precision_score(results_relation.key, results_relation.pred, average="weighted")],
             [sklearn.metrics.accuracy_score(results_relation.key, results_relation.pred)])), 
             columns = ["F1", "Recall", "Precision", "Accuracy"])

2023-12-12 14:42:13.843826: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


,F1,Recall,Precision,Accuracy
0,0.530699,0.535714,0.600818,0.535714
